In [ ]:
import torch
import torchvision
from PIL import Image
import requests
import torchvision.transforms as T
torch.__version__

In [ ]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
m = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
m.eval();

In [ ]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
im = Image.open(requests.get(url, stream=True).raw)

In [ ]:
# mean-std normalize the input image (batch-size: 1)
data = transform(im).unsqueeze(0)

# propagate through the model
outputs = m(data)

# keep only predictions with 0.7+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]


In [ ]:
import time
import numpy as np
from tqdm import tqdm

import torch._dynamo
torch._dynamo.reset()

#mode = None # 58, 55
#mode = "default" # 56, 55
#mode = "max-autotune" # 57, 55
mode = "reduce-overhead" # 58, 55
print("Compiling...")
m_opt = torch.compile(m, mode=mode)
res = m_opt(data)
print("Compilation Done")

for model, desc in [(m, "detr-resnet-50"), (m_opt, "detr-resnet-50_inductor_reduce-overhead")]:
    N = 3; i = 0
    while i < N:
        res = model(data)
        i += 1

    N = 10; i = 0
    TT = []
    while i < N:
        t0 = time.time()
        res = model(data)
        dur = (time.time() - t0) * 1000
        TT.append(dur)
        i += 1

    print(f"{desc},{np.mean(TT):.3f},{np.percentile(TT, 50):.3f}")